In [1]:
import pandas as pd, numpy as np, os
from get_draws.api import get_draws
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
from matplotlib.backends.backend_pdf import PdfPages

In [2]:
output_dir ='/ihme/costeffectiveness/results/vivarium_ciff_sam/v6.0.0_vicious_cycle/ciff_sam/2022_02_17_15_25_21/count_data/'


In [3]:
figname = 'model_6.0.0'

In [4]:
os.listdir(output_dir)

['wasting_transition_count.hdf',
 'wasting_transition_count.csv',
 'wasting_state_person_time.hdf',
 'deaths.csv',
 'wasting_state_person_time.csv',
 'deaths.hdf',
 'disease_transition_count.csv',
 'births.csv',
 'population.csv',
 'stunting_state_person_time.hdf',
 'ylds.csv',
 'ylls.csv',
 'disease_state_person_time.csv',
 'population.hdf',
 'ylls.hdf',
 'ylds.hdf',
 'disease_state_person_time.hdf',
 'births.hdf',
 'disease_transition_count.hdf',
 'stunting_state_person_time.csv']

In [5]:
wasting_pt = pd.read_csv(output_dir +'wasting_state_person_time.csv')
wasting_pt.head()

,Unnamed: 0,sex,year,cause,measure,input_draw,scenario,value,diarrhea,x_factor,sq_lns,mam_treatment,sam_treatment,age
0,0,female,2022,mild_child_wasting,state_person_time,29,baseline,0.0,cat2,cat2,covered,covered,covered,early_neonatal
1,1,female,2022,mild_child_wasting,state_person_time,29,baseline,0.0,cat1,cat2,covered,covered,covered,early_neonatal
2,2,female,2022,mild_child_wasting,state_person_time,29,baseline,0.0,cat2,cat1,covered,covered,covered,early_neonatal
3,3,female,2022,mild_child_wasting,state_person_time,29,baseline,0.0,cat1,cat1,covered,covered,covered,early_neonatal
4,4,female,2022,mild_child_wasting,state_person_time,29,baseline,0.0,cat2,cat2,covered,uncovered,covered,early_neonatal


In [6]:
def calculate_child_growth_exposure_prevalence(risk, groupby_cols):
    state_pt = pd.read_csv(output_dir +f'{risk}_state_person_time.csv')
    state_pt = state_pt.loc[state_pt.scenario=='baseline']
    state_pt = state_pt.groupby(groupby_cols).sum()[['value']].reset_index()
    if risk=='wasting':
        state_pt_exposed = state_pt.loc[state_pt.cause.str.contains('acute_malnutrition')]
        state_pt_exposed['cause'] = 'wasted'
    else:
        state_pt_exposed = state_pt.loc[state_pt.cause.isin(['cat1','cat2'])]
        state_pt_exposed['cause'] = 'stunted'
    state_pt_with_exposed = (pd.concat([state_pt, 
                          state_pt_exposed.groupby(groupby_cols).sum().reset_index()],
                         ignore_index=True)
                .set_index(groupby_cols))
    prev = state_pt_with_exposed / state_pt.groupby([c for c in groupby_cols if c != 'cause']).sum()
    #prev = prev.groupby([c for c in groupby_cols if c != 'input_draw']).describe(percentiles=[0.025,0.975]).sort_index()
    return prev

In [7]:
w = calculate_child_growth_exposure_prevalence('wasting', ['input_draw','sex','age','cause','diarrhea']).reset_index()
pr = (w.loc[w.diarrhea=='cat1'].set_index(['input_draw','sex','age','cause'])[['value']]
     / w.loc[w.diarrhea=='cat2'].set_index(['input_draw','sex','age','cause'])[['value']]).groupby(['sex','age','cause']).describe(percentiles=[0.025,0.975])
pr

value                      \
                                                    count      mean       std   
sex    age             cause                                                    
female 1-5_months      mild_child_wasting             8.0  1.080400  0.343651   
                       moderate_acute_malnutrition    6.0  1.111530  0.671019   
                       severe_acute_malnutrition     12.0  0.998315  0.003492   
                       susceptible_to_child_wasting   4.0  1.169937  0.196044   
                       wasted                        12.0  0.998434  0.003072   
       12_to_23_months mild_child_wasting            12.0  1.060778  0.007882   
                       moderate_acute_malnutrition   12.0  1.671378  0.067693   
                       severe_acute_malnutrition     12.0  1.073484  0.030567   
                       susceptible_to_child_wasting  12.0  0.901603  0.004423   
                       wasted                        12.0  1.433313  0.082367   
       2_to_4          mild_child_wasting            12.0  1.077220  0.005552   
                       moderate_acute_malnutrition   12.0  1.813560  0.021356   
                       severe_acute_malnutrition     12.0  1.195012  0.026828   
                       susceptible_to_child_wasting  12.0  0.914249  0.002685   
                       wasted                        12.0  1.724607  0.020930   
       6-11_months     mild_child_wasting            12.0  0.964654  0.016265   
                       moderate_acute_malnutrition   12.0  1.431553  0.202912   
                       severe_acute_malnutrition     12.0  1.012208  0.020044   
                       susceptible_to_child_wasting  12.0  0.897296  0.009489   
                       wasted                        12.0  1.106879  0.086135   
       early_neonatal  mild_child_wasting            12.0  0.398260  0.484538   
                       moderate_acute_malnutrition   12.0  0.465269  0.450210   
                       severe_acute_malnutrition     12.0  1.039115  0.140997   
                       susceptible_to_child_wasting  12.0  0.581153  0.511808   
                       wasted                        12.0  1.052071  0.059750   
       late_neonatal   mild_child_wasting            10.0  0.855406  0.588505   
                       moderate_acute_malnutrition   10.0  0.747706  0.468411   
                       severe_acute_malnutrition     12.0  1.003163  0.025325   
                       susceptible_to_child_wasting   9.0  0.923717  0.174666   
                       wasted                        12.0  1.007210  0.008802   
male   1-5_months      mild_child_wasting             7.0  1.388886  1.023862   
                       moderate_acute_malnutrition   10.0  1.023000  0.403712   
                       severe_acute_malnutrition     12.0  0.999173  0.003356   
                       susceptible_to_child_wasting   3.0  1.042621  1.028462   
                       wasted                        12.0  0.999071  0.003029   
       12_to_23_months mild_child_wasting            12.0  1.063228  0.010284   
                       moderate_acute_malnutrition   12.0  1.631248  0.038577   
                       severe_acute_malnutrition     12.0  1.086413  0.033604   
                       susceptible_to_child_wasting  12.0  0.877192  0.006324   
                       wasted                        12.0  1.407072  0.050316   
       2_to_4          mild_child_wasting            12.0  1.075017  0.004025   
                       moderate_acute_malnutrition   12.0  1.756599  0.023517   
                       severe_acute_malnutrition     12.0  1.191458  0.029493   
                       susceptible_to_child_wasting  12.0  0.904254  0.003368   
                       wasted                        12.0  1.654188  0.022709   
       6-11_months     mild_child_wasting            12.0  0.968408  0.011000   
                       moderate_acute_malnutrition   12.0  1.433816  0.182701   
  

In [8]:
def compute_wasting_transition_rate(transition_count_parameter, person_time_state):
    rate = (((wasting_transitions.loc[wasting_transitions.measure==f'{transition_count_parameter}']
           .set_index(['input_draw','age']).drop(columns='measure'))
          / (wasting_person_time.loc[wasting_person_time.cause==f'{person_time_state}']
             .set_index(['input_draw','age']).drop(columns='cause')))
          .groupby(['age']).describe(percentiles=[0.025,0.975]).reset_index())
    #rate = rate.loc[rate.age.isin(ages)]
    #rate['age_start'] = np.where(rate.age=='6-11_months',0.5,
    #                            np.where(rate.age=='12_to_23_months', 1.0, 2.0))
    #rate['age_end'] = np.where(rate.age=='6-11_months',1.0,
    #                            np.where(rate.age=='12_to_23_months', 2.0, 5.0))
    rate['source_state'] = person_time_state.replace('_event_count','')
    rate['sink_state'] = transition_count_parameter
    return rate#.set_index(['sex','age'])[['value']]

In [9]:
wasting_transitions = pd.read_csv(output_dir +'wasting_transition_count.csv')
wasting_person_time = pd.read_csv(output_dir +'wasting_state_person_time.csv')

In [18]:
under_six_mo = wasting_transitions.loc[wasting_transitions.age.isin(['early_neonatal',
                                                                    'late_neonatal',
                                                                    '1-5_months'])]
non_zero = under_six_mo.loc[under_six_mo['value']!=0]
non_zero

,Unnamed: 0,sex,year,measure,input_draw,scenario,value,diarrhea,x_factor,sq_lns,mam_treatment,sam_treatment,age
16,16,female,2022,mild_child_wasting_to_moderate_acute_malnutrit...,29,baseline,391.0,cat2,cat2,uncovered,covered,covered,early_neonatal
17,17,female,2022,mild_child_wasting_to_moderate_acute_malnutrit...,29,baseline,2.0,cat1,cat2,uncovered,covered,covered,early_neonatal
18,18,female,2022,mild_child_wasting_to_moderate_acute_malnutrit...,29,baseline,363.0,cat2,cat1,uncovered,covered,covered,early_neonatal
20,20,female,2022,mild_child_wasting_to_moderate_acute_malnutrit...,29,baseline,2277.0,cat2,cat2,uncovered,uncovered,covered,early_neonatal
21,21,female,2022,mild_child_wasting_to_moderate_acute_malnutrit...,29,baseline,9.0,cat1,cat2,uncovered,uncovered,covered,early_neonatal
...,...,...,...,...,...,...,...,...,...,...,...,...,...
644950,644950,male,2026,susceptible_to_child_wasting_to_mild_child_was...,946,wasting_treatment,405.0,cat2,cat1,uncovered,uncovered,covered,early_neonatal
644952,644952,male,2026,susceptible_to_child_wasting_to_mild_child_was...,946,wasting_treatment,1116.0,cat2,cat2,uncovered,covered,uncovered,early_neonatal
644954,644954,male,2026,susceptible_to_child_wasting_to_mild_child_was...,946,wasting_treatment,347.0,cat2,cat1,uncovered,covered,uncovered,early_neonatal
644956,644956,male,2026,susceptible_to_child_wasting_to_mild_child_was...,946,wasting_treatment,509.0,cat2,cat2,uncovered,uncovered,uncovered,early_neonatal


In [19]:
non_zero.measure.unique()

array(['mild_child_wasting_to_moderate_acute_malnutrition_event_count',
       'moderate_acute_malnutrition_to_severe_acute_malnutrition_event_count',
       'susceptible_to_child_wasting_to_mild_child_wasting_event_count'],
      dtype=object)

In [20]:
non_zero.age.unique()

array(['early_neonatal', 'late_neonatal', '1-5_months'], dtype=object)

In [21]:
non_zero.scenario.unique()

array(['baseline', 'lbwsg_interventions', 'sqlns', 'wasting_treatment'],
      dtype=object)

In [22]:
non_zero.diarrhea.unique()

array(['cat2', 'cat1'], dtype=object)

In [11]:
transitions = ['moderate_acute_malnutrition_to_severe_acute_malnutrition_event_count']
cause = 'moderate_acute_malnutrition'
transition_count = (wasting_transitions.loc[wasting_transitions.measure.isin(transitions)]
                    .set_index(['scenario','input_draw','age','sex','diarrhea'])[['value']])
state_pt = (wasting_person_time.loc[wasting_person_time.cause==cause]
           .set_index(['scenario','input_draw','age','sex','diarrhea'])[['value']])
rate = (transition_count / state_pt).reset_index()
rate = rate.loc[(rate.scenario=='baseline')&(rate.age.isin(['6-11_months','12_to_23_months','2_to_4']))]
rate_d = rate.loc[rate.diarrhea=='cat1'].set_index(['scenario','input_draw','age','sex'])[['value']]
rate_s = rate.loc[rate.diarrhea=='cat2'].set_index(['scenario','input_draw','age','sex'])[['value']]
rr = rate_d / rate_s
rr.groupby(['scenario','age','sex']).describe(percentiles=[0.025,0.975])

# uh-oh....
# could be a result of the weird SAM issue at initialization?

value                                \
                                 count      mean       std  min 2.5%   
scenario age             sex                                           
baseline 12_to_23_months female  480.0       inf       NaN  0.0  0.0   
                         male    479.0       inf       NaN  0.0  0.0   
         2_to_4          female  479.0  4.021612  3.752385  0.0  0.0   
                         male    480.0  3.902213  2.679409  0.0  0.0   
         6-11_months     female  476.0       inf       NaN  0.0  0.0   
                         male    479.0       inf       NaN  0.0  0.0   

                                                                 
                                      50%      97.5%        max  
scenario age             sex                                     
baseline 12_to_23_months female  3.372008  19.057811        inf  
                         male    3.639477  11.600096        inf  
         2_to_4          female  3.550687  12.459726  60.310782  
                         male    3.627477   9.435600  32.160763  
         6-11_months     female  3.958388  20.543520        inf  
                         male    4.044737  14.440077        inf

In [12]:
transitions = ['mild_child_wasting_to_moderate_acute_malnutrition_event_count']
cause = 'mild_child_wasting'
transition_count = (wasting_transitions.loc[wasting_transitions.measure.isin(transitions)]
                    .set_index(['scenario','input_draw','age','sex','diarrhea'])[['value']])
state_pt = (wasting_person_time.loc[wasting_person_time.cause==cause]
           .set_index(['scenario','input_draw','age','sex','diarrhea'])[['value']])
rate = (transition_count / state_pt).reset_index()
rate = rate.loc[(rate.scenario=='baseline')&(rate.age.isin(['6-11_months','12_to_23_months','2_to_4']))]
rate_d = rate.loc[rate.diarrhea=='cat1'].set_index(['scenario','input_draw','age','sex'])[['value']]
rate_s = rate.loc[rate.diarrhea=='cat2'].set_index(['scenario','input_draw','age','sex'])[['value']]
rr = rate_d / rate_s
rr.groupby(['scenario','sex','age']).describe(percentiles=[0.025,0.975])


# 14-18 validation target :) 

value                                 \
                                 count       mean       std       min   
scenario sex    age                                                     
baseline female 12_to_23_months  480.0  15.471167  2.685369  6.972023   
                2_to_4           480.0  16.674036  1.953241  8.817206   
                6-11_months      480.0  17.594828  4.347043  4.698936   
         male   12_to_23_months  480.0  13.825032  2.240956  7.448711   
                2_to_4           480.0  15.583095  1.616436  9.000559   
                6-11_months      480.0  15.004872  3.372011  3.791014   

                                                                             
                                      2.5%        50%      97.5%        max  
scenario sex    age                                                          
baseline female 12_to_23_months  11.099030  15.177963  21.396691  35.377604  
                2_to_4           13.099091  16.463626  21.149378  24.096049  
                6-11_months      10.362917  17.284680  27.017979  50.364780  
         male   12_to_23_months   9.776164  13.596697  19.184735  24.833025  
                2_to_4           12.504635  15.421009  19.185726  22.043116  
                6-11_months       9.232681  14.555222  22.586506  37.848061

In [13]:
transitions = ['susceptible_to_child_wasting_to_mild_child_wasting_event_count']
cause = 'susceptible_to_child_wasting'
transition_count = (wasting_transitions.loc[wasting_transitions.measure.isin(transitions)]
                    .set_index(['scenario','input_draw','age','sex','diarrhea'])[['value']])
state_pt = (wasting_person_time.loc[wasting_person_time.cause==cause]
           .set_index(['scenario','input_draw','age','sex','diarrhea'])[['value']])
rate = (transition_count / state_pt).reset_index()
rate = rate.loc[(rate.scenario=='baseline')&(rate.age.isin(['6-11_months','12_to_23_months','2_to_4']))]
rate_d = rate.loc[rate.diarrhea=='cat1'].set_index(['scenario','input_draw','age','sex'])[['value']]
rate_s = rate.loc[rate.diarrhea=='cat2'].set_index(['scenario','input_draw','age','sex'])[['value']]
rr = rate_d / rate_s
rr.groupby(['scenario','sex','age']).describe(percentiles=[0.025,0.975])


# 3.8 - 4.2 validation target :) 

value                                \
                                 count      mean       std       min   
scenario sex    age                                                    
baseline female 12_to_23_months  480.0  4.011004  0.408255  2.764507   
                2_to_4           480.0  3.811287  0.244371  2.852247   
                6-11_months      480.0  4.534176  0.714629  2.008706   
         male   12_to_23_months  480.0  4.288633  0.439089  2.655664   
                2_to_4           480.0  3.969112  0.243984  2.948188   
                6-11_months      480.0  4.716541  0.689655  2.844962   

                                                                         
                                     2.5%       50%     97.5%       max  
scenario sex    age                                                      
baseline female 12_to_23_months  3.186688  3.993733  4.923953  5.758612  
                2_to_4           3.323115  3.806652  4.293638  5.180960  
                6-11_months      3.167034  4.528967  6.110990  7.454712  
         male   12_to_23_months  3.472432  4.277044  5.197904  7.021436  
                2_to_4           3.484133  3.955683  4.513495  4.956696  
                6-11_months      3.270671  4.704701  6.230948  7.277762

In [14]:
t = wasting_transitions.reset_index()
t.loc[(t.age.isin(['early_neonatal','late_neonatal','1-5_months']))
     &(t['value']!=0)].diarrhea.unique()

array(['cat2', 'cat1'], dtype=object)